In [1]:
!pip install nltk==3.7

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
%pip install seaborn

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 292 kB 24.0 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Weights and Bias to version the dataset
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 1.8 MB 24.3 MB/s eta 0:00:01
     |████████████████████████████████| 145 kB 27.3 MB/s eta 0:00:01
     |████████████████████████████████| 181 kB 32.0 MB/s eta 0:00:01
     |████████████████████████████████| 63 kB 43.7 MB/s eta 0:00:01
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21503 sha256=195d92bb6a82422db03875291da39b43e9d213dd0554090cf7f8710f896f3310
  Stored in directory: /tmp/pip-ephem-wheel-cache-gd45t5rj/wheels/54/aa/01/724885182f93150035a2a91bce34a12877e8067a97baaf5dc8
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=d6a261bfd8028a0b38a8ad93cf71dc46d41beb60a43ad1c116e37198f5bbc15e
  Stored in directory: /tmp/pip-ephem-wheel-cache-gd45t5rj/wheels/4c/8e/7e/72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
Successfully built promise pathtools


In [4]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="darkgrid")
import warnings
warnings.filterwarnings("ignore")

In [6]:
import wandb

In [7]:
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [8]:
# Use the following if you don't have the dataset in your system
# To download the dataset from wandb artifacts
# run = wandb.init()
# artifact = run.use_artifact('alokpadhi/Medical-Transcription/processed_dataset:v0', type='raw_data')
# artifact_dir = artifact.download()

In [9]:
import ipywidgets as widgets

In [10]:
df = pd.read_csv("../../dataset/processed/processed_transcriptions.csv")
df.head()

,medical_specialty,transcription
0,allergy / immunology,subjective 23 year old white female presents c...
1,bariatrics,past medical history difficulty climbing stair...
2,bariatrics,history present illness seen abc today pleasan...
3,cardiovascular / pulmonary,2 mode 1 left atrial enlargement left atrial d...
4,cardiovascular / pulmonary,1 left ventricular cavity size wall thickness ...


In [11]:
# Feature and target
X = df.transcription.to_numpy()
y = df.medical_specialty

In [12]:
import json

In [13]:
# Label encoder to encode class labels
class LabelEncoder(object):
    """Encode labels into unqiue ids/integers"""
    def __init__(self, class_to_index={}):
        self.class_to_index = class_to_index or {}
        self.index_to_class = {v:k for k, v in self.class_to_index.items()}
        self.classes = list(self.class_to_index.keys())
        
    def __len__(self):
        return len(self.class_to_index)
    
    def __str__(self):
        return f"<LabelEncoder(num_classes={len(self)})>"
    
    def fit(self, y):
        classes = np.unique(y)
        for i, class_ in enumerate(classes):
            self.class_to_index[class_] = i
        self.index_to_class = {v:k for k, v in self.class_to_index.items()}
        self.classes = list(self.class_to_index.keys())
        return self
    
    def encode(self, y):
        encoded = np.zeros(len(y), dtype=int)
        for i, item in enumerate(y):
            encoded[i] = self.class_to_index[item]
        return encoded
    
    def decode(self, y):
        classes = []
        for i, item in enumerate(y):
            classes.append(self.index_to_class[item])
            
        return classes
    
    def save(self, fp):
        with open(fp, "w") as fp:
            contents = {"class_to_index": self.class_to_index}
            json.dump(contents, fp, indent=4, sort_keys=False)
    
    @classmethod
    def load(cls, fp):
        with open(fp, "r") as fp:
            kwargs = json.load(fp=fp)
        return cls(**kwargs)

In [22]:
label_encoder = LabelEncoder.load("../../data_artifacts/label_encoder.json")

In [23]:
label_encoder.class_to_index

{'allergy / immunology': 0,
 'autopsy': 1,
 'bariatrics': 2,
 'cardiovascular / pulmonary': 3,
 'chiropractic': 4,
 'consult history phy': 5,
 'cosmetic / plastic surgery': 6,
 'dentistry': 7,
 'dermatology': 8,
 'diets nutritions': 9,
 'discharge summary': 10,
 'emergency room reports': 11,
 'endocrinology': 12,
 'ent otolaryngology': 13,
 'gastroenterology': 14,
 'general medicine': 15,
 'hematology oncology': 16,
 'hospice palliative care': 17,
 'ime qme work comp etc': 18,
 'lab medicine pathology': 19,
 'letters': 20,
 'nephrology': 21,
 'neurology': 22,
 'neurosurgery': 23,
 'obstetrics / gynecology': 24,
 'office notes': 25,
 'ophthalmology': 26,
 'orthopedic': 27,
 'pain management': 28,
 'pediatrics neonatal': 29,
 'physical medicine rehab': 30,
 'podiatry': 31,
 'psychiatry / psychology': 32,
 'radiology': 33,
 'rheumatology': 34,
 'sleep medicine': 35,
 'soap / chart / progress notes': 36,
 'speech language': 37,
 'surgery': 38,
 'urology': 39}

In [24]:
NUM_CLASSES = len(label_encoder)

In [25]:
# Check one example
print(f"Target sample: {y[15]}")
print(f"Encoded target sample: {label_encoder.encode([y[15]])}")

Target sample: bariatrics
Encoded target sample: [2]


In [26]:
# Encode all our labels
y = label_encoder.encode(y)
print(y.shape)

(4966,)


In [27]:
from sklearn.model_selection import train_test_split

In [28]:
# split sizes
train_size = 0.7
val_size = 0.15
test_size = 0.15

In [29]:
# To ensure the target distribution remains same across the splits
X_train, X_, y_train, y_ = train_test_split(
                            X, y, train_size=train_size, stratify=y)

In [30]:
print(f"train: {len(X_train)} ({(len(X_train)/len(X)):.2f})\n"
     f"remaining: {len(X_)} ({(len(X_)/len(X)):.2f})")

train: 3476 (0.70)
remaining: 1490 (0.30)


In [32]:
# split for validation and test set
X_val, X_test, y_val, y_test = train_test_split(
                                X_, y_, train_size=0.5, stratify=y_)

In [33]:
print(f"train: {len(X_train)} ({(len(X_train)/len(X)):.2f})\n"
     f"val: {len(X_val)} ({(len(X_val)/len(X)):.2f})\n"
     f"test: {len(X_test)} ({(len(X_test)/len(X)):.2f})")

train: 3476 (0.70)
val: 745 (0.15)
test: 745 (0.15)


In [35]:
# Get counts for each class
counts = {}
counts["train_counts"] = {specialty: label_encoder.decode(y_train).count(specialty) for specialty in label_encoder.classes}
counts["val_counts"] = {specialty: label_encoder.decode(y_val).count(specialty) for specialty in label_encoder.classes}
counts["test_counts"] = {specialty: label_encoder.decode(y_test).count(specialty) for specialty in label_encoder.classes}

In [38]:
pd.set_option("display.max_columns", 40)

In [39]:
# convert the class distribution across splits to dataframe
pd.DataFrame({
    "train": counts["train_counts"],
    "val": counts["val_counts"],
    "test": counts["test_counts"]
}).T.fillna(0)

,allergy / immunology,autopsy,bariatrics,cardiovascular / pulmonary,chiropractic,consult history phy,cosmetic / plastic surgery,dentistry,dermatology,diets nutritions,discharge summary,emergency room reports,endocrinology,ent otolaryngology,gastroenterology,general medicine,hematology oncology,hospice palliative care,ime qme work comp etc,lab medicine pathology,letters,nephrology,neurology,neurosurgery,obstetrics / gynecology,office notes,ophthalmology,orthopedic,pain management,pediatrics neonatal,physical medicine rehab,podiatry,psychiatry / psychology,radiology,rheumatology,sleep medicine,soap / chart / progress notes,speech language,surgery,urology
train,5,6,13,260,10,361,19,19,20,7,76,52,13,67,157,181,63,4,11,6,16,57,156,66,108,35,58,248,43,49,15,33,37,191,7,14,116,6,762,109
val,1,1,3,56,2,78,4,4,4,2,16,11,3,15,33,39,13,1,3,1,3,12,33,14,23,8,13,53,9,10,3,7,8,41,2,3,25,2,163,23
test,1,1,2,55,2,77,4,4,5,1,16,12,3,14,34,39,14,1,2,1,4,12,34,14,24,7,12,54,9,11,3,7,8,41,1,3,25,1,163,24


In [40]:
# adjust counts across splits due to the propertion difference between them
for k in counts["val_counts"].keys():
    counts["val_counts"][k] = int(counts["val_counts"][k] * \
                                 (train_size/val_size))
    
for k in counts["test_counts"].keys():
    counts["test_counts"][k] = int(counts["test_counts"][k] * \
                                  (train_size/test_size))

In [41]:
dist_df = pd.DataFrame({
    "train": counts["train_counts"],
    "val": counts["val_counts"],
    "test": counts["test_counts"]
}).T.fillna(0)
dist_df

,allergy / immunology,autopsy,bariatrics,cardiovascular / pulmonary,chiropractic,consult history phy,cosmetic / plastic surgery,dentistry,dermatology,diets nutritions,discharge summary,emergency room reports,endocrinology,ent otolaryngology,gastroenterology,general medicine,hematology oncology,hospice palliative care,ime qme work comp etc,lab medicine pathology,letters,nephrology,neurology,neurosurgery,obstetrics / gynecology,office notes,ophthalmology,orthopedic,pain management,pediatrics neonatal,physical medicine rehab,podiatry,psychiatry / psychology,radiology,rheumatology,sleep medicine,soap / chart / progress notes,speech language,surgery,urology
train,5,6,13,260,10,361,19,19,20,7,76,52,13,67,157,181,63,4,11,6,16,57,156,66,108,35,58,248,43,49,15,33,37,191,7,14,116,6,762,109
val,4,4,14,261,9,364,18,18,18,9,74,51,14,70,154,182,60,4,14,4,14,56,154,65,107,37,60,247,42,46,14,32,37,191,9,14,116,9,760,107
test,4,4,9,256,9,359,18,18,23,4,74,56,14,65,158,182,65,4,9,4,18,56,158,65,112,32,56,252,42,51,14,32,37,191,4,14,116,4,760,112


In [42]:
# check the standard deviation
np.mean(np.std(dist_df.to_numpy(), axis=0))

1.2239856098339892

In [44]:
train_df = pd.DataFrame({"transcription": X_train, "specialty": label_encoder.decode(y_train)})
val_df = pd.DataFrame({"transcription": X_val, "specialty": label_encoder.decode(y_val)})
test_df = pd.DataFrame({"transcription": X_test, "specialty": label_encoder.decode(y_test)})

In [45]:
train_df.shape

(3476, 2)

In [46]:
val_df.shape

(745, 2)

In [47]:
test_df.shape

(745, 2)

In [48]:
# check train
train_df.head()

,transcription,specialty
0,indications 55 year old female colonoscopy scr...,surgery
1,axial back pain greatly improved completely er...,consult history phy
2,preoperative diagnoses epiretinal membrane rig...,ophthalmology
3,preoperative diagnosis squamous cell carcinoma...,surgery
4,exam ultrasound guided paracentesis history as...,surgery


In [49]:
from pathlib import Path

In [59]:
train_dataset_path = Path("../../dataset/v1.0/train/train.parquet")
val_dataset_path = Path("../../dataset/v1.0/val/val.parquet")
test_dataset_path = Path("../../dataset/v1.0/test/test.parquet")

In [60]:
type(train_dataset_path)

pathlib.PosixPath

In [61]:
# store the above splits both in parquet and csv format
# parquet format is a column based, and faster than csv to read
train_df.to_parquet(train_dataset_path,index=False)
val_df.to_parquet(val_dataset_path,index=False)
test_df.to_parquet(test_dataset_path,index=False)

In [66]:
v1_datapath = Path("../../dataset/v1.0/")

In [69]:
# dataset version for v1
datav1_run = wandb.init(project="Medical-Transcription")
datav1 = wandb.Artifact("Dataset_V1_0", type="raw_data", description="Train, Val and test splits", metadata={"train_size": len(train_df), "val_size": len(val_df),
                                                                                                            "test_size": len(test_df)})
for dir in ["train", "val", "test"]:
    datav1.add_dir(Path(v1_datapath, dir))
    
datav1_run.log_artifact(datav1)

wandb: Adding directory to artifact (./../../dataset/v1.0/train)... Done. 0.1s
wandb: Adding directory to artifact (./../../dataset/v1.0/val)... Done. 0.0s
wandb: Adding directory to artifact (./../../dataset/v1.0/test)... Done. 0.0s


In [1]:
# # use this code to download the split dataset files
# import wandb
# run = wandb.init()
# artifact = run.use_artifact('alokpadhi/Medical-Transcription/Dataset_V1_0:v0', type='raw_data')
# artifact_dir = artifact.download()